In [ ]:
from importlib import reload

In [ ]:
import classes

In [ ]:
classes = reload(classes)

In [ ]:
art = classes.Curator()

In [ ]:
obs = await art.get_sample('landscape')

In [ ]:
art.collections['landscape'].cache

In [ ]:
art.collections['watercolor'].cache

In [ ]:
start_ts = classes.time()
await art.get_object(obs[110])
print(classes.time()-start_ts)

---

In [1]:
import requests
import os

In [18]:
req = requests.get(r"https://images-api.nasa.gov/search", 
                   params={'q':'nebula',
                           'media_type':'image'})

In [19]:
req

<Response [200]>

In [20]:
data = req.json()

In [23]:
data['collection']['items']

[{'href': 'https://images-assets.nasa.gov/image/PIA14417/collection.json',
  'links': [{'href': 'https://images-assets.nasa.gov/image/PIA14417/PIA14417~thumb.jpg',
    'rel': 'preview',
    'render': 'image'}],
  'data': [{'secondary_creator': 'NASA/JPL-Caltech/Harvard-Smithsonian CfA',
    'description_508': 'The Dumbbell nebula, also known as Messier 27, pumps out infrared light in this image from NASA Spitzer Space Telescope. Planetary nebulae are now known to be the remains of stars that once looked a lot like our sun.',
    'nasa_id': 'PIA14417',
    'date_created': '2011-08-10T21:00:09Z',
    'keywords': ['Helix Nebula', 'Spitzer Space Telescope'],
    'title': 'Weighing in on the Dumbbell Nebula',
    'center': 'JPL',
    'media_type': 'image',
    'description': 'The Dumbbell nebula, also known as Messier 27, pumps out infrared light in this image from NASA Spitzer Space Telescope. Planetary nebulae are now known to be the remains of stars that once looked a lot like our sun.'}